In [24]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

Use $w=(2,-3.4)^T$, $b=4.2$, and noise $\varepsilon$ to synthesize data via $$y=Xw+b+\varepsilon$$

In [25]:
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w))) # normal distribution
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [26]:
print(f"features: {features[0]},\nlabel: {labels[0]}")

features: tensor([1.0828, 0.5261]),
label: tensor([4.5829])


In [27]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

batch_size = 10
for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[-1.6474, -0.0856],
        [ 0.9333,  0.0892],
        [-2.1847,  2.0167],
        [-0.7234,  0.5702],
        [-0.7851,  0.6533],
        [ 0.0902, -0.5198],
        [ 0.0322, -1.0030],
        [ 0.8179, -0.2194],
        [ 0.7234, -0.2799],
        [ 1.4318, -0.0044]]) 
 tensor([[ 1.2102],
        [ 5.7846],
        [-7.0235],
        [ 0.8194],
        [ 0.3985],
        [ 6.1421],
        [ 7.6784],
        [ 6.5739],
        [ 6.5852],
        [ 7.0714]])


In [29]:
# Initialize parameters
w = torch.normal(0, 0.01, size=(2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

def linreg(X, w, b):
    return torch.matmul(X, w) + b

# Loss function
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

# Optimizer
def sgd(params, lr, batch_size):
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [30]:
# Training
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        l.sum().backward()
        sgd([w, b], lr, batch_size)
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.041445
epoch 2, loss 0.000159
epoch 3, loss 0.000050
